# Project 3 - Eniac (Data Cleaning & Storytelling)

## 1. Overview

## 2. Function declaration & DataFrames creation

In [ ]:
import pandas as pd

def gd_path(file_id):
    """Generate a shareable link from Google Drive file id."""
    return f"https://drive.google.com/uc?export=download&id={file_id}"


# Google Drive file ids
file_ids = {
    "brands": "1GNoGAkphYToef9d3ODV4OychFVu5dl6Y",
    "orderlines": "1Me5gWCAUsnA19ixP6NrG1pkzvSOCWeEW",
    "orders": "1S-PLC3SzKBfhuXSXPvdFsSSy391XeKlf",
    "products": "1e1M8hbD9CNoomB-ttq_gvzgmWTUNmQk2",
}

# Read data from Google Drive
brands_df = pd.read_csv(gd_path(file_ids["brands"]), sep=",")
orderlines_df = pd.read_csv(gd_path(file_ids["orderlines"]), sep=",")
orders_df = pd.read_csv(gd_path(file_ids["orders"]), sep=",")
products_df = pd.read_csv(gd_path(file_ids["products"]), sep=",")

## 3. Some Info on the DataFrames


In [ ]:
print(brands_df.head(),"\n")
print(brands_df.describe(),"\n")
print(brands_df.nunique(),"\n")

In [ ]:
print(orderlines_df.head(),"\n")
print(orderlines_df.describe(),"\n")
print(orderlines_df.nunique(),"\n")

In [ ]:
print(orders_df.head(),"\n")
print(orders_df.describe(),"\n")
print(orders_df.nunique(),"\n")

In [ ]:
print(products_df.head(),"\n")
print(products_df.describe(),"\n")
print(products_df.nunique(),"\n")

### Business questions

1.	How should products be classified into different categories to simplify reports and analysis?
2.	What is the distribution of product prices across different categories?
3.	How many products are being discounted?
4.	How big are the offered discounts as a percentage of the product prices?
5.	How do seasonality and special dates (Christmas, Black Friday) affect sales?
6.	How could data collection be improved?


### Practical questions / tasks

1.	Assess data quality of the dataset provided
2.	Undertake data cleaning tasks
3.	Make data usable and trustable (with the necessary disclaimer)

## 4. Data Cleaning `orders_df`, `orderlines_df`, and `products_df`

### 4.1. `orders_df`

In [ ]:
orders_df.duplicated().sum()

0

In [ ]:
orders_df["created_date"] = pd.to_datetime(orders_df["created_date"])

In [ ]:
orders_df = orders_df.loc[~orders_df.total_paid.isna(), :].copy()
# Deletes missing values (Jenny)

In [ ]:
orders_df.info()

### 4.2. `orderlines_df`

In [ ]:
orderlines_df.duplicated().sum()

0

In [ ]:
orderlines_df["date"] = pd.to_datetime(orderlines_df["date"])

In [ ]:
# DO NOT RUN unless you'd like to see the whole error message
orderlines_df["unit_price"] = pd.to_numeric(orderlines_df["unit_price"])
# ValueError: Unable to parse string "1.137.99" at position 6

In [ ]:
# counts the values affected by the above error message
orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+").value_counts()

In [ ]:
# gives you the percentage of the values affected by the above error
two_dot_percentage = ((orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+").value_counts()[1] / orderlines_df.shape[0])*100).round(2)
print(f"The 2 dot problem represents {two_dot_percentage}% of the rows in our DataFrame")

In [ ]:
# creates a list with the 2 dot problem values
two_dot_order_ids_list = orderlines_df.loc[orderlines_df.unit_price.str.contains("\d+\.\d+\.\d+"), "id_order"]
# drops the values from the list created above
orderlines_df = orderlines_df.loc[~orderlines_df.id_order.isin(two_dot_order_ids_list)]

In [ ]:
# now the column can be converted to numeric
orderlines_df["unit_price"] = pd.to_numeric(orderlines_df["unit_price"])

In [ ]:
orderlines_df.info()

### 4.3. `products_df`

Checking `desc`, `type` and `price`

In [ ]:
products_df.info()#.sample(50)

In [ ]:
products_df.drop_duplicates(inplace=True)
# drops all duplicates (Jenny) (Nursah)

In [ ]:
products_df

In [ ]:
#gives you the number of NaNs in "type"
products_df["type"].isna().sum()

In [ ]:
# gets rid if the NaNs in "type"
products_df.dropna(subset=['type'], inplace=True)
# df.dropna(subset=[column_name], inplace=True)

In [ ]:
# gives you the number of NaNs in "desc"
products_df["desc"].isna().sum()

In [ ]:
# gets rid if the NaNs in "desc"
products_df.dropna(subset=['desc'], inplace=True)

In [ ]:
# gives you the number of NaNs in "price"
products_df["price"].isna().sum()

In [ ]:
# gets rid if the NaNs in "price"
products_df.dropna(subset=['price'], inplace=True)

In [ ]:
products_df.isnull().sum()
# checks if there are missing values in the products table (Jenny)

In [ ]:
# removes all rows with 2 dot & decimals points followed by 3 digits values in them DataFrame
products_df = products_df.loc[(~products_df.price.astype(str).str.contains("\d+\.\d+\.\d+"))&(~products_df.price.astype(str).str.contains("\d+\.\d{3,}")), :]

In [ ]:
# sets "price" column to numeric
products_df["price"] = pd.to_numeric(products_df["price"])

Checking `promo_price`

In [ ]:
# checks the column "promo_price" for 2 dot and 3 digit problems and
promo_problems_number = products.loc[(products.promo_price.astype(str).str.contains("\d+\.\d+\.\d+"))|(products.promo_price.astype(str).str.contains("\d+\.\d{3,}")), :].shape[0]
# prints the number and the percentage of the values affected relative to the column
print(f"The column promo_price has in total {promo_problems_number} wrong values. This is {round(((promo_problems_number / products.shape[0]) * 100), 2)}% of the rows of the DataFrame")

Since the two problem seem to have affected over 90% of the values in column `promo_price`, we suggest to drop the whole column.

In [ ]:
# gets rid of the whole column "promo_price"
products_df = products.drop(columns=["promo_price"])
# another way of doing this is:
products_df = products_df.drop("promo_price", axis=1)

In [ ]:
# Alternative removes the 1st dot
products_df["promo_price"] = products_df["promo_price"].str.replace(r'(\.\d+)\.', r'\1')

Check the info on `products_df`, `orders_df`, and `orderlines_df`

In [ ]:
orders_df.info()

In [ ]:
orderlines_df.info()

In [ ]:
products_df.info()

## 5. Download Cleaned DataFrames

In [ ]:
from google.colab import files

#orders_df.to_csv("orders_cl.csv", index=False)
#files.download("orders_cl.csv")

#orderlines_df.to_csv("orderlines_cl.csv", index=False)
#files.download("orderlines_cl.csv")

products_df.to_csv("products_cl.csv", index=False)
files.download("products_cl.csv")

---
<font color="red"> The code of the above cell will download the cleaned datasets with the following names:</font>
* `brands_cl.csv`
* `orderlines_cl.csv`
* `orders_cl.csv`
* `products_cl.csv`

---